In [10]:
from flask import Flask, request, jsonify
from pymongo import MongoClient
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [11]:
app = Flask(__name__)

# MongoDB connection setup
client = MongoClient('mongodb://localhost:27017/')
db = client['studyzen']
collection = db['study_groups']

In [12]:
# Define mappings
mappings = {
    "group_size": {'Small (2-4 members)': 1, 'Medium (5-8 members)': 2, 'Large (9+ members)': 3},
    "interaction_style": {
        'Informal and spontaneous discussions': 1,
        'Structured and goal-oriented sessions': 2,
        'Regular meetings with set agendas': 3
    },
    "study_schedule": {
        'Early morning sessions': 1,
        'Late morning sessions': 2,
        'Afternoon sessions': 3,
        'Evening sessions': 4,
        'Flexible scheduling': 5
    },
    "session_length": {
        'Short (30-60 minutes)': 1,
        'Medium (1-2 hours)': 2,
        'Long (2+ hours)': 3
    },
    "engagement_level": {
        'Low engagement (occasional study sessions)': 1,
        'Flexible engagement (variable based on need)': 2,
        'Moderate engagement (few times a week)': 3,
        'High engagement (daily sessions, active participation)': 4
    },
    "leadership_style": {
        'No formal leader (collaborative leadership)': 1,
        'Rotational leadership (different leaders per session)': 2,
        'Fixed leader (one consistent leader)': 3
    },
    "group_support": {
        'Peer accountability and motivation': 1,
        'Academic support and tutoring': 2,
        'Emotional support and stress management': 3
    }
}

In [13]:
@app.route('/recommend', methods=['POST'])
def recommend():
    user_preferences = request.json

    # Convert user preferences to numerical form
    user_vector = np.array([
        mappings['group_size'][user_preferences['preferred_group_size']],
        mappings['study_schedule'][user_preferences['preferred_study_schedule']],
        mappings['interaction_style'][user_preferences['preferred_interaction_style']],
        mappings['session_length'][user_preferences['preferred_study_session_length']],
        mappings['engagement_level'][user_preferences['engagement_level']],
        mappings['leadership_style'][user_preferences['leadership_style']],
        mappings['group_support'][user_preferences['group_support']]
    ]).reshape(1, -1)

    # Fetch study groups from MongoDB
    study_groups = list(collection.find({}, {'_id': 0, 'groupName': 1, 'groupSize': 1, 'studySchedule': 1,
                                             'interactionStyle': 1, 'sessionLength': 1, 'groupGoals': 1,
                                             'engagementLevel': 1, 'leadershipStyle': 1, 'groupSupport': 1}))

    # Convert study groups to numerical form
    study_groups_numeric = []
    for group in study_groups:
        study_groups_numeric.append([
            mappings['group_size'][group['groupSize']],
            mappings['study_schedule'][group['studySchedule']],
            mappings['interaction_style'][group['interactionStyle']],
            mappings['session_length'][group['sessionLength']],
            mappings['engagement_level'][group['engagementLevel']],
            mappings['leadership_style'][group['leadershipStyle']],
            mappings['group_support'][group['groupSupport']]
        ])
    study_groups_numeric = np.array(study_groups_numeric)

    # k-NN implementation
    knn = NearestNeighbors(n_neighbors=5, metric='cosine')
    knn.fit(study_groups_numeric)
    distances, indices = knn.kneighbors(user_vector)

    recommended_groups = [study_groups[idx] for idx in indices[0]]
    return jsonify(recommended_groups)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1